The objective of this notebook is to fix the party abbreviations in `MP` excel file to correspond to `parties` table.

In [24]:
import pandas as pd
from pathlib import Path
from typing import List, Union

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

directory = ".."
mp_path = Path(directory) / Path("Croatia_MPs_final_ 20220917.xlsx")
parties_path = Path(directory) / Path("Croatia_parties_final_20220917.xlsx")

mpdf = pd.read_excel(str(mp_path))#.dropna()
partiesdf = pd.read_excel(str(parties_path))

# import pandas as pd
# mpdf = pd.read_pickle("mpdf")
# partiesdf = pd.read_pickle("partiesdf")



# Let's filter parties that do not appear in the metadata:

In [26]:
import pandas as pd
from pathlib import Path
from typing import List, Union

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

directory = ".."
mp_path = Path(directory) / Path("Croatia_MPs_final_ 20220917.xlsx")
parties_path = Path(directory) / Path("Croatia_parties_final_20220917.xlsx")

mpdf = pd.read_excel(str(mp_path))
partiesdf = pd.read_excel(str(parties_path))

def is_present(query: str, contents=None) -> bool:
    paths = list(
        Path("/home/rupnik/parlamint/").glob("*_meta.tsv"))
    if not contents:
        for path in paths:
            with open(str(path)) as f:
                if query in f.read():
                    return True
    else:
        for c in contents:
            if query in c:
                return True
    return False

contents = []
paths = list(Path("/home/rupnik/parlamint/").glob("*_meta.tsv"))
for p in paths:
    with open(str(p)) as f:
        contents.append(f.read())

from tqdm import tqdm
tqdm.pandas()

partiesdf["party_present"] = partiesdf.party.progress_apply(lambda s: is_present(f'{s}', contents=contents))
partiesdf["full_name_present"] = partiesdf.full_name.progress_apply(lambda s: is_present(f'{s}', contents=contents))
mpdf["speaker_present"] = mpdf.fullname.progress_apply(lambda s: is_present(f'{s}', contents=contents))
mpdf.to_pickle("mpdf")
partiesdf.to_pickle("partiesdf")

100%|██████████| 1385/1385 [00:48<00:00, 28.71it/s]


Let's drop all parties that do not appear in the metadata neither with their fullname neither with their abbreviation. Also let's drop speakers that do not appear in the metadata or that spoke in Term 4:

In [27]:
partiesdf = partiesdf[partiesdf.party_present | partiesdf.full_name_present].reset_index(drop=True)
mpdf = mpdf[mpdf.speaker_present & (mpdf.term2 != 4)].reset_index(drop=True)

Let's see which values do we have to impute:

In [28]:
values_to_fill = set(mpdf.party[mpdf.speaker_present]) - set(partiesdf.party)
values_to_fill

{'Blok',
 'Centar',
 'Damir Bajs NL',
 'Domovinski pokret',
 'HKS',
 'HSP dr. Ante Starčević',
 'Hrvatski suverenisti',
 'Most',
 'Možemo!',
 'NL',
 'NZ',
 'RF',
 'SDA HR',
 'SsIP',
 nan,
 'nezavisni'}

We can already change NZ to nezavisni and solve one of them:

In [29]:
mpdf["party"] = mpdf.party.replace({
    "NZ": "nezavisni"
})
values_to_fill = set(mpdf.party[mpdf.speaker_present]) - set(partiesdf.party)

Approach:

For every discrepancy in party abbreviations we look in the respective term and see which party the person belongs to.

In [30]:
from typing import Union
import pandas as pd
mpdf["party_new"] = mpdf.party
from functools import lru_cache

@lru_cache
def get_party(name: str, term: Union[str, int]) -> str:
    df = pd.read_csv(f"/home/rupnik/parlamint/{term}_meta.tsv", sep="\t", usecols=["Term", "Speaker_name", "Speaker_party"]).drop_duplicates()
    subset = df[df.Speaker_name == name]
    subset = subset[subset.Term == int(term)]
    
    if subset.shape[0] == 1:
        return subset.Speaker_party.values[0]
    else:
        return subset.Speaker_party

In [31]:
mpdf["party_new"] = mpdf.party.copy()
mpdf = mpdf.loc[mpdf.speaker_present & (mpdf.term2 != 4)]
mpdf = mpdf.reset_index(drop=True)


for i, row in mpdf.iterrows():
    if row["party_new"] not in values_to_fill:
        continue
    try:
        mpdf.loc[i, "party_new"] = get_party(row["fullname"], row["term2"])
    except:
        continue

In [32]:
mpdf.loc[mpdf.party_new.isna(),:]

,codemp,order_id,term1,term2,term_id,type_of_list,fullname,firstname,lastname,party,date_of_birth,year_of_birth,gender,place_of_birth,field_of_study,education_y,constituency,bp_lat,bp_lon,speaker_present,party_new
943,M660,1180,2016-2020,9,185,not_active,"Marić, Zdravko",Zdravko,Marić,nezavisni,19770203,1977,0,Slavonski Brod,4,22,5,45.163143,18.011608,True,NaN
983,M335,1224,2020-2023,10,35,not_active,"Bubalo, Krešimir",Bubalo,Krešimir,NaN,19731105,1973,0,Osijek,4,16,4,45.554962,18.695514,True,NaN
1058,M660,1300,2020-2023,10,111,not_active,"Marić, Zdravko",Marić,Zdravko,nezavisni,19770203,1977,0,Slavonski Brod,4,22,5,45.163143,18.011608,True,NaN


In [33]:
#Set Marić, Zdravko to nezavisni:
mpdf.loc[(mpdf.fullname == "Marić, Zdravko") & mpdf.party_new.isna(),"party_new"] = "nezavisni"

# Set Bubalo, Krešimir to DP (according to wikipedia)
mpdf.loc[(mpdf.fullname == "Bubalo, Krešimir") & mpdf.party_new.isna(),"party_new"] = "DP"

mpdf.loc[mpdf.party_new.isna(),:]


,codemp,order_id,term1,term2,term_id,type_of_list,fullname,firstname,lastname,party,date_of_birth,year_of_birth,gender,place_of_birth,field_of_study,education_y,constituency,bp_lat,bp_lon,speaker_present,party_new


In [34]:
mpdf.loc[mpdf.party_new.isin(values_to_fill), "party_new"].unique()

array(['nezavisni', 'Most', 'HKS', 'Hrvatski suverenisti'], dtype=object)

Nezavisni is to be left alone. I can find `MOST` in the metadata and parties excel, so I can use that.

I can't find HKS anywhere (not in the metadata, nor in parties, but I can find it in coalition_composition attributes: `partiesdf[partiesdf.coalition_composition.str.lower().str.contains("hks")]`)

Same goes with `Hrvatski suverenisti`: I can't find `suverenisti` nor `HS`  in metadata. For now I'll just convert them to "HSu", because there is no parties with this abbreviation yet.

In [35]:
partiesdf[partiesdf.party == "MOST"].sample()

,codeparty,term1,term2,party,full_name,established,chairman,ideology_LR,party_family,election_result,no_seats,coalition,coalition_composition,ruling,party_present,full_name_present
62,P63,2015-2016,8,MOST,Most nezavisnih lista,2012,Božo Petrov,4,1,13.64,19,0,-,1,True,True


In [36]:
mpdf["party_new"] = mpdf.party_new.replace({
    "Most": "MOST",
    "Hrvatski suverenisti": "HS",
    "Nezavisni": "nezavisni",
})

In [37]:
mpdf["party"] = mpdf.party_new
mpdf = mpdf.drop(columns=["party_new", 'speaker_present'])
mpdf.to_pickle("mpdf_corrected")
partiesdf.to_pickle("partiesdf_corrected")